In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from mongo import attraction_db
from model.google import Google
from model.Spider import Spider 
from model.Word import Word 
from bs4 import BeautifulSoup
import pandas as pd
import json
import time
import csv
import re
import os

In [20]:
def sort_by_weekday(operating_hours):
    days_order = ['星期一', '星期二', '星期三', '星期四', '星期五', '星期六', '星期日']
    sorted_hours = sorted(operating_hours, key=lambda x: days_order.index(x['week']))
    return sorted_hours

def get_place_more_info(driver):
    try:
        data={}
        page_content = driver.page_source
        soup = BeautifulSoup(page_content, 'html.parser')
        open_time_week = []
        try:data["subtitle"] = soup.select('.DkEaL')[0].text
        except:data["subtitle"] = ""
        try:data["info"] = soup.select('.PYvSYb')[0].text
        except:data["info"] = ""
        try:data["site"] = soup.select('a.CsEnBe')[0].get('href')
        except:data["site"] = ""
        try:
            open_time = soup.select('.y0skZc')
            for time_up in open_time:
                week = time_up.select('td div')[0].text
                clock = time_up.select('td.mxowUb .G8aQO')[0].text
                result={}
                result['week'] = week
                result['clock'] = clock
                open_time_week.append(result)
            data["open_time"] = sort_by_weekday(open_time_week)
        except:
            open_time = []
            data["open_time"] = open_time
        try:
            library = driver.find_element(By.CSS_SELECTOR,'.aoRNLd.kn2E5e.NMjTrf.lvtCsd')
            library.click()
            time.sleep(0.5)
            data["photo"] = get_photo_library(driver)
        except:
            data["photo"] = []
    except:
        data["subtitle"] = ""
        data["info"] = ""
        data["site"] = ""
        data["open_time"] = []
        data["photo"] = []
    return data

def get_photo(style):
    url_match = re.search(r'url\((.+)\);', style)
    if url_match:
        extracted_url = url_match.group(1)
        return extracted_url
    
def multiply_wh_parameters(original_url, factor):
    def modify_size(match):
        return f"{match.group(1)}{int(match.group(2)) * factor}"
    
    url_parts = original_url.split('=')

    if len(url_parts) > 1:
        modified_params = re.sub(r'([wh])(\d+)', modify_size, url_parts[1])
        modified_url = url_parts[0] + '=' + modified_params
    else:
        modified_url = original_url
    
    return modified_url

def get_photo_library(driver):
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    style_element = soup.find_all('div', class_='U39Pmb')

    photo_library = []

    for element in style_element:
        style_attribute = element['style']
        url = get_photo(style_attribute)
        url_pattern = re.compile(r'https?://(?:www\.)?[\w\.-]+(?:\.[a-zA-Z]{2,6})+[\w\.-]*[\?#]?[\w\.-=&]*')
        url = url.replace('"',"")
        if re.match(url_pattern, url):
            photo_library.append(multiply_wh_parameters(url,10))

    return photo_library


In [4]:
url = 'https://www.google.com/maps/place/%E5%85%AB%E9%87%8C%E5%8D%81%E4%B8%89%E8%A1%8C%E6%96%87%E5%8C%96%E5%85%AC%E5%9C%92/@25.1520013,121.3953676,13.18z/data=!4m6!3m5!1s0x3442a51740d10deb:0x521fe6ec74e2cebe!8m2!3d25.1651782!4d121.4132327!16s%2Fg%2F11bbrls82z?authuser=0&entry=ttu'

In [21]:
driver = webdriver.Chrome()
driver.set_window_size(800,800)
driver.get(url)

data = get_place_more_info(driver)
print(data)
driver.quit()

{'subtitle': '公園', 'info': '這座公園有滑草坡和大型沙坑，是放風箏的熱門去處。', 'site': 'https://www.hrcm.ntpc.gov.tw/Home/Page?page_id=764', 'open_time': [{'week': '星期一', 'clock': '24 小時營業'}, {'week': '星期二', 'clock': '24 小時營業'}, {'week': '星期三', 'clock': '24 小時營業'}, {'week': '星期四', 'clock': '24 小時營業'}, {'week': '星期五', 'clock': '24 小時營業'}, {'week': '星期六', 'clock': '24 小時營業'}, {'week': '星期日', 'clock': '24 小時營業'}], 'photo': ['https://lh5.googleusercontent.com/p/AF1QipPPY_0Pj2wLTfSyNBRTPDO1E0EFcRypHUBKNqVF=w2030-h1350-k-no', 'https://lh5.googleusercontent.com/p/AF1QipOZEHlc5PkWNY-5YwSnbw9wjH6gUW8U3CrJNRf-=w2030-h1140-k-no', 'https://lh5.googleusercontent.com/p/AF1QipPfyKwd8gdJoMjVERqrEOPx28LJ9UiMFO0T4G7b=w2030-h1140-k-no', 'https://lh5.googleusercontent.com/p/AF1QipPOeguv2t14-3h64wXC_rmllz3mqOqWXKrPJraL=w2030-h1520-k-no', 'https://lh5.googleusercontent.com/p/AF1QipOiJ9D_f_WL9t5tu-XgPrMfymTw5J0FJ-T5VP-m=w2030-h2700-k-no']}


In [ ]:
'https://lh5.googleusercontent.com/p/AF1QipPPY_0Pj2wLTfSyNBRTPDO1E0EFcRypHUBKNqVF=w203-h135-k-no'
'https://lh50.googleusercontent.com/p/AF10QipPPY_0Pj20wLTfSyNBRTPDO10E0EFcRypHUBKNqVF=w2030-h1350-k-no'